# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
#pip install docarray

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
response = index.query(query)

In [10]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, wrinkle resistant, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. The Men's Plaid Trop

## Step By Step

In [11]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [12]:
docs = loader.load()

In [13]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [15]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [16]:
print(len(embed))

1536


In [17]:
print(embed[:5])

[-0.021923632663435447, 0.006714269791913915, -0.018236133678182616, -0.03919069729085128, -0.014057805358445133]


In [18]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [23]:
retriever = db.as_retriever()

In [24]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [25]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [26]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [27]:
display(Markdown(response))

| Shirt Name | Description | Sun Protection |
| --- | --- | --- |
| Sun Shield Shirt | High-performance sun shirt that is slightly fitted and falls at the hip. Made from 78% nylon and 22% Lycra Xtra Life fiber. Wicks moisture for quick-drying comfort and is abrasion resistant. | SPF 50+, blocks 98% of the sun's harmful rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection shirt designed for hot weather and extended travel. Made from 52% polyester and 48% nylon. Features front and back cape venting and two front bellows pockets. | UPF 50+, blocks 98% of the sun's harmful rays |
| Men's TropicVibe Shirt, Short-Sleeve | Lightweight sun-protection shirt with a traditional fit. Made from 71% Nylon and 29% Polyester. Features front and back cape venting and two front bellows pockets. | UPF 50+, blocks 98% of the sun's harmful rays |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with a traditional fit. Made from 100% polyester. Features front and back cape venting and two front bellows pockets. | SPF 50+, blocks 98% of the sun's harmful rays |

In [28]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [29]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [30]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
display(Markdown(response))

Sure, here is a summary of each shirt with sun protection:

| Product ID | Name | Description | Material | Sun Protection | Additional Features |
| --- | --- | --- | --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | A light hot-weather shirt with a traditional fit. It is wrinkle-resistant and has front and back cape venting. | 100% Polyester | UPF 50+ | Two front bellows pockets |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | A shirt originally designed for fishing, it is wrinkle-free and quickly evaporates perspiration. | 52% Polyester, 48% Nylon | UPF 50+ | Front and back cape venting, two front bellows pockets |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | A lightweight sun-protection shirt with a traditional fit. It is wrinkle resistant. | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. | UPF 50+ | Front and back cape venting, two front bellows pockets |
| 255 | Sun Shield Shirt | A high-performance sun shirt that wicks moisture for quick-drying comfort. It fits comfortably over your favorite swimsuit. | 78% Nylon, 22% Lycra Xtra Life fiber | UPF 50+ | Abrasion resistant |

In [32]:
response = index.query(query, llm=llm)

In [33]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.